##### (1) For every row of the table, calibrate the SABR model using the Hagan et. al. approximation for the implied volatility. Use a fixed β of 1. Construct all 5 strikes of the market instruments. Report in a neat table the strikes, implied volatilites at those strikes, and parameters of the SABR model.

In [26]:
%pylab
import numpy as np
import pandas as pd
import scipy.optimize as opt
from scipy.stats import norm
from scipy.optimize import fsolve
import pdb


Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


//anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['norm', 'beta']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [27]:
S=3.724 #The spot rate is 3.724 BRL per USD
rf=0.022# usd interest rate=rf
rd=0.065
CALL=1
PUT=-1 #use in w

In [28]:
Tenors = ['ON', '1W', '2W', '1M', '2M', '3M', '6M', '1Y']
Expiry = np.array([1, 7, 14, 30, 60, 90, 180, 360])/360
ATM = np.array([20.98, 13.91, 13.75, 14.24, 13.84, 13.82, 13.82, 13.94])/ 100
RR_25d = np.array([1.2, 1.3, 1.4, 1.5, 1.75, 2.0, 2.4, 2.9])/ 100
BF_25d = np.array([0.15, 0.20, 0.20, 0.22, 0.27, 0.32, 0.43, 0.55]) / 100

Volatility = pd.DataFrame(np.array([Expiry, ATM, RR_25d, BF_25d]).T, index=Tenors, columns=['Expiry', 'ATM', 'RR', 'BF'])
Volatility

,Expiry,ATM,RR,BF
ON,0.002778,0.2098,0.0120,0.0015
1W,0.019444,0.1391,0.0130,0.0020
2W,0.038889,0.1375,0.0140,0.0020
1M,0.083333,0.1424,0.0150,0.0022
2M,0.166667,0.1384,0.0175,0.0027
3M,0.250000,0.1382,0.0200,0.0032
6M,0.500000,0.1382,0.0240,0.0043
1Y,1.000000,0.1394,0.0290,0.0055


$$
K_{ATM}=Se^{-\frac{1}{2} \sigma^{2}T+\left(R_{d}-R{f}\right)T} 
$$


In [29]:
def ATM_K(T,vol):
    'Try to find the Strike in ATM'
    return np.exp(-0.5*(vol**2)*T+(rd-rf)*T)*S




$$
d_{2}=\frac{\log (S / K)+\left(R_{d}-R_{f}-\frac{1}{2} \sigma \right)T}{\sigma \sqrt{T}}
$$

$$
delta_{Call}=\frac{K}{S}e^{-r_{f}T}N\left(d_{2}\right)
$$
$$
delta_{Put}=-\frac{K}{S}e^{-r_{f}T}N\left(-d_{2}\right)
$$

In [30]:
def get_delta(sigma,K,T,w):
    'Try to get the delta'
    d2 = (np.log(S / K) + (rd - rf - .5 * sigma * sigma) * T) / (sigma*np.sqrt(T))
    return w * K * np.exp(-rf * T) / S * norm.cdf(w * d2)


In [31]:
def Solve_K(delta, T, vol, w,k0):

    def get_delta_diff(K):
        'Try to find optimized K'
        d2 = (np.log(S/K) + (rd-rf-.5*(vol**2))*T) / (vol*np.sqrt(T))
        return w * K * np.exp(-rd*T) / S * norm.cdf(w*d2) - delta

    return fsolve(get_delta_diff, k0)


In [32]:
ATM_K = np.array([ATM_K(T=t, vol=a) for a, t in zip(ATM, Expiry)]).flatten()
ATM_K


array([3.72421716, 3.72641393, 3.7288615 , 3.73421188, 3.74480225,
       3.75527288, 3.78680838, 3.85003477])

In [33]:

K_BFPut = np.array([Solve_K(-0.25, t, v,PUT,S) for t, v in zip(Expiry, ATM + BF_25d)]).flatten()

K_BFPut


array([3.69658999, 3.678049  , 3.66181241, 3.63403487, 3.60884045,
       3.59064525, 3.55921071, 3.5368504 ])

In [34]:

K_BFCall = np.array([Solve_K(0.25, t, v, CALL,S) for t, v in zip(Expiry, ATM + BF_25d)]).flatten()

K_BFCall


array([3.75253806, 3.77692458, 3.80010537, 3.84404044, 3.89906627,
       3.94736798, 4.06978408, 4.27707391])

$$
\begin{array}{l}
\sigma_{I V}\left(K, t_{e}\right) \\
\begin{array}{c}
\simeq \frac{\alpha}{(F K)^{(1-\beta) / 2\left\{1+\frac{(1-\beta)^{2}}{24} \log ^{2} F / K+\frac{(1-\beta)^{4}}{1920} \log ^{4} F / K\right\}}} \\
\qquad \begin{array}{c}
\hat{\alpha}(0)=\alpha \\
\qquad\left(\frac{z}{x(z)}\right) \cdot\left\{1+\left[\frac{(1-\beta)^{2}}{24} \frac{\alpha^{2}}{(F K)^{1-\beta}}+\frac{1}{4} \frac{\rho \beta \nu \alpha}{(F K)^{(1-\beta) / 2}}+\frac{2-3 \rho^{2}}{24} \nu^{2}\right] t_{e}\right\}
\end{array} \\
z=\frac{\nu}{\alpha}(F K)^{(1-\beta) / 2} \log F / K \\
x(z)=\log \left(\frac{\sqrt{1-2 \rho z+z^{2}}+z-\rho}{1-\rho}\right)
\end{array}
\end{array}
$$

In [35]:
def SABR_vol(a, v, rho, K, T, b):

    F = S * np.exp((rd - rf) * T)
    z = v / a * (F * K) ** ((1 - b) / 2) * np.log(F / K)
    x = np.log((np.sqrt(1 - 2 * rho * z + z ** 2) + z - rho) / (1 - rho))
    vol = a / ((F * K) ** ((1 - b) / 2) * (
            1 + (1 - b) ** 2 / 24 * (np.log(F / K)) ** 2 + (1 - b) ** 4 / 1920 * (np.log(F / K)) ** 4)) * (z / x) * (
                      1 + ((1 - b) ** 2 / 24 * a ** 2 / (F * K) ** (1 - b) + 1 / 4 * rho * b * v * a / (F * K) ** (
                      (1 - b) / 2) + (2 - 3 * rho ** 2) / 24 * v ** 2) * T)
    return vol

$$
d_{1}=\frac{\log (S / K)+\left(R_{d}-R_{f}+\frac{1}{2} \sigma \right)T}{\sigma \sqrt{T}}
$$
$$
d_{2}=\frac{\log (S / K)+\left(R_{d}-R_{f}-\frac{1}{2} \sigma \right)T}{\sigma \sqrt{T}}
$$

$$
BS_{call}=SN\left(d_{1}\right)-e^{-\left(r_{d}-r_{f}\right)T}KN\left(d_{2}\right)
$$

$$
BS_{Put}=-SN\left(d_{1}\right)+e^{-\left(r_{d}-r_{f}\right)T}KN\left(d_{2}\right)
$$

In [36]:
def BS_price(vol,K,T,w):
    d1 = (np.log(S / K) + (rd - rf + .5 * vol**2) * T) / (vol* np.sqrt(T))
    d2 = (np.log(S / K) + (rd - rf - .5 * vol**2) * T) / (vol *np.sqrt(T))
    return w*S*norm.cdf(w*d1)-w*np.exp(-(rd-rf)*T)*K*norm.cdf(w*d2)

$$
\begin{array}{c}
IV\left(K_{ATM}\right)=atm\\
P^{m k t}\left(K_{B F P u t}\right)+C^{m k t}\left(K_{B F C a l l}\right)= 
P^{B S}\left(A T M+B F, K_{B F P u t}\right)+C^{B S}\left(A T M+B F, K_{B F C a l l}\right) \\
IV\left(K_{R R C a l l}\right)-I V\left(K_{R R P u t}\right)=R R \\
\Delta_{C}\left(I V\left(K_{R R C a l l}, K_{R R C a l l}\right)\right)=\delta=0.25\\
\Delta_{P}\left(I V\left(K_{R R P u t}, K_{R R P u t}\right)\right)=\delta=-0.25\\
\Delta_{C}\left(I V\left(K_{ATM+BF C a l l}, K_{ATM+BF C a l l}\right)\right)=\delta=0.25\\
\Delta_{P}\left(I V\left(K_{ATM+BF P u t}, K_{ATM+BF P u t}\right)\right)=\delta=-0.25
\end{array}
$$

In [37]:
beta=1
CALL=1
PUT=-1
def Find_Market_Instruments(atm, rr, bf, atm_k, bf_call_k, bf_put_k, T):
    def solve_SABR(x):
        '''x[0] is alpha,
           x[1] is vega
           x[2] is rho
           x[3] is strike for rr_call
           x[4] is strike for rr_put'''
        
        ''' condition 1: the vol of ATM_strike under SABR should equal to market atm vol
            condition 2: the sum of BF call prcie and BF put price should equal to the sum of call and put under BS price for the samke strike
            condition 3: delta of RR call is 0.25
            condition 4: delta of RR put is -0.25
            conditiion 5: vol of RR_call -vol of RR_put=RR
            condition 6: delta of atm+bf call =0.25
            condition 7: delta of atm_bf put =-0.25
            condition 8: (vol of BF_CALL+vol of BF_PUT)/2-atm unnder SABR =bf
            
            '''
        
        beta=1
        
        return np.array([SABR_vol(x[0], x[1], x[2], atm_k, T, beta) - atm, #1
                         
                         BS_price(SABR_vol(x[0], x[1], x[2], bf_call_k, T, beta),bf_call_k, T, CALL) - 
                         BS_price(atm + bf, bf_call_k, T, CALL)+
                         BS_price(SABR_vol(x[0], x[1], x[2], bf_put_k, T, beta), bf_put_k, T, PUT)-
                         BS_price(atm + bf, bf_put_k, T, PUT),#2
                        
                         get_delta(SABR_vol(x[0], x[1], x[2], x[3], T, beta), x[3], T, CALL) - 0.25,#3

                         get_delta(SABR_vol(x[0], x[1], x[2], x[4], T, beta), x[4], T, PUT) + 0.25,#4
 
                         SABR_vol(x[0], x[1], x[2], x[3], T, beta) - SABR_vol(x[0], x[1], x[2], x[4], T, beta) - rr,#5
                         
                         get_delta(atm+bf,bf_call_k,T,CALL)-0.25,#6
                         get_delta(atm+bf,bf_put_k,T,PUT)+0.25,#7
                         
                         #(SABR_vol(x[0], x[1], x[2],bf_call_k,T,beta)+SABR_vol(x[0], x[1], x[2],bf_put_k,T,beta))/2 - SABR_vol(x[0], x[1], x[2], atm_k, T, beta)-bf #8
             
                         ]) **2
    

    return opt.least_squares(solve_SABR, x0=np.array([.1, 2, 0.1, 4, 3]),
                             bounds=([0, 0, -1, 0, 0], [np.inf, np.inf, 1, np.inf, np.inf])).x

w


In [38]:

data_1=np.array([Find_Market_Instruments(ATM[i], RR_25d[i], BF_25d[i], ATM_K[i], K_BFCall[i], K_BFPut[i], Expiry[i]) for i in range(8)])
data_1

array([[0.20920253, 4.42066202, 0.34822492, 3.75327175, 3.69731123],
       [0.13898494, 2.48446245, 0.37665962, 3.77936988, 3.68012267],
       [0.13861421, 1.51136797, 0.46848326, 3.8041921 , 3.66453728],
       [0.13765693, 2.14046501, 0.23369525, 3.85111658, 3.63791147],
       [0.13730913, 1.37597735, 0.25778313, 3.91274235, 3.611996  ],
       [0.13391545, 1.2405287 , 0.31498987, 3.96441763, 3.59952578],
       [0.13086052, 1.07908504, 0.293106  , 4.10399085, 3.5707586 ],
       [0.13304268, 0.75412481, 0.35602323, 4.34768215, 3.55189213]])

In [39]:
alpha_set=[]
vega_set=[]
rho_set=[]
K_rr_call_set=[]
K_rr_Put_set=[]
for i in data_1:
    alpha_set.append(i[0])
    vega_set.append(i[1])
    rho_set.append(i[2])
    K_rr_call_set.append(i[3])
    K_rr_Put_set.append(i[4])

In [40]:
vol_atm=np.array([SABR_vol(alpha_set[i],vega_set[i],rho_set[i],ATM_K[i],Expiry[i],1) for i in range(8)])
vol_bf_call=np.array([SABR_vol(alpha_set[i],vega_set[i],rho_set[i],K_BFCall[i],Expiry[i],1) for i in range(8)])
vol_bf_put=np.array([SABR_vol(alpha_set[i],vega_set[i],rho_set[i],K_BFPut[i],Expiry[i],1) for i in range(8)])
vol_rr_call=np.array([SABR_vol(alpha_set[i],vega_set[i],rho_set[i],K_rr_call_set[i],Expiry[i],1) for i in range(8)])
vol_rr_put=np.array([SABR_vol(alpha_set[i],vega_set[i],rho_set[i],K_rr_Put_set[i],Expiry[i],1) for i in range(8)])


In [41]:
Beta_set=[1. for i in range(8)]
Beta_set

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [42]:
Expiry_list = [1/360, 7/360, 14/360, 30/360, 60/360, 90/360, 180/360, 360/360]

result=pd.DataFrame({'Time':Expiry_list,"Alpha":alpha_set,"Beta":Beta_set,"Vega":vega_set,"Rho":rho_set,"ATM_K":ATM_K,"K_RR_Call":K_rr_call_set,
                    "K_RR_Put":K_rr_Put_set,"K_BF_Call":K_BFCall,"K_BF_Put":K_rr_Put_set,"Vol_ATM":vol_atm,
                    "Vol_RR_Call":vol_rr_call,"Vol_RR_Put":vol_rr_put,"Vol_BF_Call":vol_bf_call,"Vol_BF_Put":vol_bf_put},index=Tenors)

In [43]:
result

,Time,Alpha,Beta,Vega,Rho,ATM_K,K_RR_Call,K_RR_Put,K_BF_Call,K_BF_Put,Vol_ATM,Vol_RR_Call,Vol_RR_Put,Vol_BF_Call,Vol_BF_Put
ON,0.002778,0.209203,1.0,4.420662,0.348225,3.724217,3.753272,3.697311,3.752538,3.697311,0.209976,0.216684,0.205103,0.216499,0.204993
1W,0.019444,0.138985,1.0,2.484462,0.376660,3.726414,3.779370,3.680123,3.776925,3.680123,0.140079,0.147736,0.135227,0.147345,0.135061
2W,0.038889,0.138614,1.0,1.511368,0.468483,3.728862,3.804192,3.664537,3.800105,3.664537,0.139304,0.147047,0.133766,0.146602,0.133561
1M,0.083333,0.137657,1.0,2.140465,0.233695,3.734212,3.851117,3.637911,3.844040,3.637911,0.141661,0.153555,0.139042,0.152654,0.139103
2M,0.166667,0.137309,1.0,1.375977,0.257783,3.744802,3.912742,3.611996,3.899066,3.611996,0.140554,0.151727,0.137267,0.150625,0.137266
3M,0.250000,0.133915,1.0,1.240529,0.314990,3.755273,3.964418,3.599526,3.947368,3.599526,0.137537,0.152035,0.132807,0.150656,0.132758
6M,0.500000,0.130861,1.0,1.079085,0.293106,3.786808,4.103991,3.570759,4.069784,3.570759,0.136309,0.155425,0.132718,0.152980,0.132839
1Y,1.000000,0.133043,1.0,0.754125,0.356023,3.850035,4.347682,3.551892,4.277074,3.551892,0.138030,0.160482,0.132487,0.156912,0.132479


In [44]:
result.to_csv("Result.csv")

(b)For every row of the table, find strikes for put delta of −10% and call delta of 10%. Create of graph of implied volatiltiy vs. strike for this range of strikes.

##### way 1:

In [45]:
alpha_10=alpha_set[-1]
vega_10=vega_set[-1]
rho_10=rho_set[-1]



In [46]:
def solve_K_10(delta,w):
    def K_function_10(K):
        vol_10=SABR_vol(alpha_10,vega_10,rho_10,K,1,1)
        return Solve_K(delta,1,vol_10,w,S)-K
    return fsolve(K_function_10,S)

K_10_Call=solve_K_10(delta=0.1,w=CALL)[0]
print("K_10_Call is"+str(K_10_Call))
K_10_Put=solve_K_10(delta=-0.1,w=PUT)[0]
print("K_10_Put is"+str(K_10_Put))

K_10_Call is5.014403184873235
K_10_Put is3.2836440879579087


#####  way 2:


$$
d_{2}=\frac{\log (S / K)+\left(R_{d}-R_{f}-\frac{1}{2} \sigma \right)T}{\sigma \sqrt{T}}
$$

$$
delta_{Call}=\frac{K}{S}e^{-r_{f}T}N\left(d_{2}\right)
$$
$$
delta_{Put}=-\frac{K}{S}e^{-r_{f}T}N\left(-d_{2}\right)
$$

In [47]:
def get_delta_d(K):
    sigma=SABR_vol(alpha, vega,rho,K,T,1)

    d2 = (np.log(S / K) + (rd - rf - .5 * sigma * sigma) * T) / (sigma*np.sqrt(T))
    return w * K * np.exp(-rf * T) / S * norm.cdf(w * d2)-w*0.1


In [48]:
c=[]
p=[]
for i in range(8):
    alpha=alpha_set[i]
    vega=vega_set[i]
    rho=rho_set[i]
    T=Expiry_list[i]
    w=1
    K_call = fsolve(get_delta_d,S)[0]
    c.append(K_call)
    w=-1
    K_put = fsolve(get_delta_d,S)[0]
    p.append(K_put)
    strikes = np.arange(K_put,K_call,.001)
    vols = [SABR_vol(alpha,vega,rho,j,T,1)*100 for j in strikes]
    plt.plot(strikes,vols)
plt.legend(Tenors)
plt.show()

In [23]:

print("K_10_Call is"+str(K_call))
K_10_Put=solve_K_10(delta=-0.1,w=PUT)[0]
print("K_10_Put is"+str(K_put))

K_10_Call is5.051726989545236
K_10_Put is3.2715631328211137
